In [1]:
import torch

In [17]:
A = torch.Tensor([
    [0, 1, 0, 10, 100],
    [1, 0, 0, 10, 0]
])

embeddings = torch.Tensor([
    [0, 1, 1,],
    [2, 3, 3],
    [4, 5, 5],
    [6, 7, 7],
    [8 ,9, 9],
])

def add_pad_embedding(embeddings):
    return torch.cat([embeddings, torch.zeros(1, embeddings.shape[1])])

embeddings = add_pad_embedding(embeddings)
embeddings = torch.nn.Embedding.from_pretrained(embeddings, freeze=False, padding_idx=-1)
display(embeddings)
display(embeddings.weight)
display(A)

tensor([[0., 1., 1.],
        [2., 3., 3.],
        [4., 5., 5.],
        [6., 7., 7.],
        [8., 9., 9.],
        [0., 0., 0.]])

Embedding(6, 3, padding_idx=5)

Parameter containing:
tensor([[0., 1., 1.],
        [2., 3., 3.],
        [4., 5., 5.],
        [6., 7., 7.],
        [8., 9., 9.],
        [0., 0., 0.]], requires_grad=True)

tensor([[  0.,   1.,   0.,  10., 100.],
        [  1.,   0.,   0.,  10.,   0.]])

In [33]:
'''
1. turn nonzeros into indices
2. use indices as in normal transformer to get embedding,
pretty much like token indices - should I include padding tokens?
3. but also somehow use these indices to extract the output
4. it may be that I can also accomplish this via masking
'''
def pad(vec, seq_len, pad_token=-1):
    return torch.cat([
        vec, torch.Tensor([pad_token]).expand(seq_len - vec.size()[0])
    ])

def select_nonzero(A, seq_len=10, pad_token=-1):
    seqs = []
    for row in A:
        nonzeros = pad(row.nonzero().T[0], seq_len, pad_token)
        seqs.append(nonzeros)
    return torch.stack(seqs)

In [34]:
A

tensor([[  0.,   1.,   0.,  10., 100.],
        [  1.,   0.,   0.,  10.,   0.]])

In [39]:
nonzeros = select_nonzero(A, pad_token=embeddings.weight.size()[0]-1)
nonzeros

tensor([[1., 3., 4., 5., 5., 5., 5., 5., 5., 5.],
        [0., 3., 5., 5., 5., 5., 5., 5., 5., 5.]])

torch.Size([2, 10])

In [44]:
squished_embeddings = embeddings(nonzeros.long())
squished_embeddings

tensor([[[2., 3., 3.],
         [6., 7., 7.],
         [8., 9., 9.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 1.],
         [6., 7., 7.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], grad_fn=<EmbeddingBackward>)

In [49]:
display(squished_embeddings)
display(nonzeros)

tensor([[[2., 3., 3.],
         [6., 7., 7.],
         [8., 9., 9.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 1.],
         [6., 7., 7.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], grad_fn=<EmbeddingBackward>)

tensor([[1., 3., 4., 5., 5., 5., 5., 5., 5., 5.],
        [0., 3., 5., 5., 5., 5., 5., 5., 5., 5.]])

In [62]:
squished_embeddings * nonzeros.unsqueeze(-1)

tensor([[[ 2.,  3.,  3.],
         [18., 21., 21.],
         [32., 36., 36.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [18., 21., 21.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]], grad_fn=<MulBackward0>)

In [58]:
squished_embeddings.shape

torch.Size([2, 10, 3])